In [ ]:
import pickle

In [ ]:
x = []
y = []

In [ ]:
import numpy as np
def flatten(d):    
    res = []  # Result list
    if isinstance(d, dict):
        for key, val in d.items():
            if key == 'controller' or key == 'action_counter':
                pass
            res.extend(flatten(val))
    elif isinstance(d, list):
        for i in d:
            if isinstance(i, dict):
                res.extend(flatten(i))
            else:
                res.extend(i)
    elif isinstance(d, np.ndarray):
        d = d.tolist()     
        for i in d:
            if isinstance(i, dict):
                res.extend(flatten(i))
            else:
                res.append(i)
    else:
        raise TypeError("Undefined type for flatten: %s"%type(d))

    return res

def getpair(experience):
    f = flatten(experience['state'])
    #f.extend(experience['prev_action'].tolist())
    return f, experience['action'][len(experience['action']) - 1]

In [ ]:
with open('/home/victor/ML/Smash/phillip-master/agents/FalconFalconBF/experience/320404a83c3644f6aab384a5668ed19a_102','rb') as f:
    experience = pickle.load(f)
experience
#s, a = getpair(experience)
#a

In [ ]:
import os
import keras
from sklearn import preprocessing
for root, dirs, filenames in os.walk('/home/victor/ML/Smash/phillip-master/agents/FalconFalconBF/experience/'):
    for f in filenames:
        log = open(os.path.join(root, f),'rb')
        experience = pickle.load(log)
        #print(experience['state'])
        s, a = getpair(experience)
        x.append(s)
        y.append(a)
scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)
y = keras.utils.to_categorical(y, 32)

In [ ]:
len(y)

In [ ]:
#model.save('model')

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
from keras.models import load_model


In [ ]:
m = load_model('model')

In [ ]:
m.evaluate(x_test, y_test)

In [ ]:
t = flatten(experience['state'])
t = scaler.transform(t)
m.predict(np.expand_dims(t, axis=0))

In [ ]:
from sklearn.externals import joblib
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

# And now to load...

scaler = joblib.load(scaler_filename) 

In [ ]:
len(flatten(experience['state']))

In [ ]:
experience['state']

In [ ]:
str(3)

In [9]:
import h5py
import numpy as np
from sklearn.externals import joblib
import keras
from sklearn import preprocessing

# And now| to load...

ry = []

In [10]:
def getData(scaler_filename = "scaler.save"):
    h5f = h5py.File('statedata.hdf5', 'r')
    x = []
    y = []
    for k in h5f.keys():
        x.append(h5f.get(k)[:-1])
        ry.append(int(h5f.get(k)[-1:]))
        y.append(int(h5f.get(k)[-1:]))
    scaler = preprocessing.StandardScaler().fit(x)
    x = scaler.transform(x)
    y = keras.utils.to_categorical(y, 32)
    
    joblib.dump(scaler, scaler_filename) 
    return x, y

In [11]:
scaler_filename = "scaler.save"
a, b = getData(scaler_filename)

In [13]:
s = a[0]
for i in range(9):
    s = np.append(s, a[i + 1])
x = []
y = []
for i in range(2000 - 20):
    x.append(s)
    y.append(b[i + 10])
    s = np.append(s[10:], a[i + 11])

In [6]:
x[0]

array([-1.14290222,  0.        ,  1.12215333, -0.0524464 ,  1.28233482,
        0.04564183, -0.82433745,  0.40004824,  0.        , -0.33373833,
       -0.21873933, -0.3765295 ,  0.21494066,  0.        ,  0.83233071,
        1.92081251,  0.02000362,  0.91062092, -0.02335591, -0.52766169,
        0.0584471 ,  0.        ,  0.        , -1.27736133,  0.        ,
       -0.95702727,  0.67824402,  1.12937036, -0.0581762 , -0.43128959,
       -0.80841894,  0.        , -0.16404891, -0.2160653 , -0.2578281 ,
        0.84274407, -0.14493656,  1.42930677, -0.01808614,  0.02193603,
       -0.91595711,  0.0183552 , -0.17605688,  0.39717317,  0.        ,
        0.        , -1.73132519,  0.        ,  0.        ,  0.        ,
        0.        , -0.58328903, -0.58328903,  1.04218225, -0.80398528,
        0.        ,  1.12215333,  0.4617831 , -0.45191542,  0.04564183,
       -0.66354271, -0.99379348,  0.        , -0.33373833, -0.21873933,
       -0.3765295 , -1.06027895,  0.        , -1.20144552,  1.20

In [ ]:
scaler = joblib.load(scaler_filename) 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import regularizers
from keras.optimizers import SGD
model = Sequential([
    Dense(128, input_shape=(len(x[0]),), kernel_initializer='random_uniform'),
    Activation('relu'),
    Dropout(0.3),
    Dense(128, kernel_initializer='random_uniform'),
    Activation('relu'),
    Dropout(0.3),
    Dense(128, kernel_initializer='random_uniform'),
    Activation('relu'),
    Dropout(0.3),
    Dense(128, kernel_initializer='random_uniform'),
    Activation('relu'),
    Dropout(0.3),
    Dense(128, kernel_initializer='random_uniform'),
    Activation('relu'),
    Dropout(0.3),
    Dense(32, kernel_initializer='random_uniform'),
    Activation('softmax'),
])
model.compile(optimizer=keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
np.random.seed(70) 
x_train = x[:len(x) * 4 // 5]
y_train = y[:len(y) * 4 // 5]
x_test = x[len(x) * 4 // 5:]
y_test = y[len(y) * 4 // 5:]
x_train = np.array(x_train).astype('float32')
model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_test, y_test))


In [ ]:
from keras.models import load_model
model = load_model('model')

In [ ]:
np.argmax(model.predict(np.expand_dims(x[0], axis=0)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
gaussian_numbers = np.random.randn(1000)
plt.hist(ry, bins=32)
plt.title("Gaussian Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")

fig = plt.gcf()
fig.show()

In [ ]:
import tensorflow as tf
graph = tf.Graph()
graph.finalize()
tf_config = dict(
        allow_soft_placement=True,
        #log_device_placement=True,
      )
sess = tf.Session(
        graph=graph,
        config=tf.ConfigProto(**tf_config),
      ) 
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('snapshot')

In [ ]:
variables = tf.global_variables()
saver = tf.train.Saver(variables)

In [ ]:
saver.restore(sess, 'snapshot')